<a href="https://colab.research.google.com/github/iamscj/abuse_detection/blob/main/Minor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
from torchvision import transforms
import random

In [ ]:
model = torchvision.models.resnet50(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
model.eval()

In [ ]:
# Define the transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize frames to 128x128 pixels
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
def extract_features(frame):
    frame = transform(frame)
    frame = torch.unsqueeze(frame, 0)
    with torch.no_grad():
        features = model(frame)
    return features.squeeze()

In [ ]:
import os
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
video_dir = '/content/drive/MyDrive/MINOR_DATASET/UBI_FIGHTS/fight'
output_dir = '/content/drive/MyDrive/MINOR_OUTPUT'

In [ ]:
video_files = os.listdir(video_dir)
random.shuffle(video_files)
video_files

In [ ]:
print(len(video_files))
num_frames_to_extract = 10  # Number of frames to extract from each video

216


In [ ]:
import cv2

In [ ]:
def load_frames_from_video(video_path,cnt):
    frames = []
    cap = cv2.VideoCapture(video_path)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    print(len(frames))
    return frames

In [ ]:
from PIL import Image
import time
import datetime

In [ ]:
def get_video_length(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps
    cap.release()
    return duration

In [ ]:
def generate_random_number_list(n):
    number_list = list(range(1, n))
    random.shuffle(number_list)
    return number_list[:10]

In [ ]:
def get_particular_frames(video_path, frame_numbers):
    frames = []
    cap = cv2.VideoCapture(video_path)
    for frame_number in frame_numbers:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
    cap.release()
    return frames

In [ ]:
def get_video_frame_count(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    return frame_count

In [ ]:
for video_file in video_files[100:]:
    video_path = os.path.join(video_dir, video_file)
    frame_count=get_video_frame_count(video_path)
    frame_list=generate_random_number_list(frame_count)
    # frames = load_frames_from_video(video_path,frame_count)
    frames = get_particular_frames(video_path,frame_list)
    if len(frames) < num_frames_to_extract:
        continue  # Skip videos with fewer frames than required

    selected_frames = random.sample(frames, num_frames_to_extract)
    features = []

    for frame in selected_frames:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert frame to RGB format
        pil_frame = Image.fromarray(frame)  # Convert NumPy array to PIL image
        transformed_frame = transform(pil_frame)
        transformed_frame = torch.unsqueeze(transformed_frame, 0)
        with torch.no_grad():
            frame_features = model(transformed_frame)
        features.append(frame_features.squeeze())

    features = torch.stack(features)
    output_file = os.path.join(output_dir, f'{video_file}.pt')
    torch.save(features, output_file)
    print(video_file + " trained")
    # time.sleep(5)

In [ ]:
import os
import torch

# Define the directory containing the .pt files
directory = '/content/drive/MyDrive/MINOR_OUTPUT'

# Load and concatenate the features from all .pt files
combined_features = []
for file_name in os.listdir(directory):
    if file_name.endswith('.pt'):
        file_path = os.path.join(directory, file_name)
        features = torch.load(file_path)
        combined_features.append(features)

combined_features = torch.cat(combined_features, dim=0)

# Save the combined features to a single .pt file
output_file = '/content/drive/MyDrive/MINOR_OUTPUT/model.pt'
torch.save(combined_features, output_file)
